In [19]:
import pandas as pd
import numpy as np
import math
from scipy import stats

import os

data_path = '/Users/ryan/Documents/GitHub/Finance-Empirical-Studies/Machine Learning in the Chinese Stock Market/Data'

## Variable Construction

(1) $acc = [(\Delta CA − \Delta CASH) − (\Delta CL −  \Delta STD − \Delta TP) − Dep]/Total Assets$

CA, CASH, CL, STD, TP, Dep, denote current assets, cash/cash equivalents, current liabilities, debt included in current liabilities, income tax payable, depreciation and amortization expense, respectively.

(2) $absacc$: Absolute value of $acc$

(3) $agr$: Annual percentage change in total assets.

In [35]:
# Read data
file_path = 'FI_T6/FI_T6.csv'
FI_t6 = pd.read_csv(os.path.join(data_path, file_path))

file_path = 'FS_Combas/FS_Combas.csv'
FS_combas = pd.read_csv(os.path.join(data_path, file_path))

# Process data
FI_t6 = FI_t6[FI_t6['Typrep'] == 'A']
FI_t6.rename(columns = {'F061201B': 'Dep'}, inplace=True)

FS_combas = FS_combas[FS_combas['Typrep'] == 'A']
FS_combas.rename(columns = {'A0b1103000': 'CASH', 
                    'A001100000': 'CA', 
                    'A001000000': 'TA', 
                    'A001124000': 'STD', 
                    'A002113000': 'TP', 
                    'A002100000': 'CL'},
                    inplace=True)

# Calculate characteristics
data = pd.merge(FI_t6[['Stkcd', 'Accper', 'Dep']], 
                FS_combas[['Stkcd', 'Accper', 'CASH', 'STD', 'CA', 'TA', 'TP', 'CL']],
                on = ['Stkcd', 'Accper'],
                how = 'left')

data.fillna(0, inplace=True)
#! Needs interpretation here: Can I use 0 to simply replace the null?

data['acc'] = ((data['CA'] - data['CA'].shift(1))-
                (data['CASH'] - data['CASH'].shift(1))-
                ((data['CL'] - data['CL'].shift(1))-
                (data['STD'] - data['STD'].shift(1))-
                (data['TP'] - data['TP'].shift(1)))-
                data['Dep'])/\
                data['TA']

# factor values
data['absacc'] = abs(data['acc'])

data['agr'] = (data['TA'] - data['TA'].shift(1)) / data['TA'].shift(1)

(4) $beta$: We estimate stock-level beta using weekly returns and value-weighted market returns for three years ending month $t − 1$ with at least 52 weeks of returns.

(5) $betasq$: Stock-level market beta squared

In [ ]:
# read files
file_path = 'TRD_Weekm/TRD_Weekm.csv'
TRD_w = pd.read_csv(os.path.join(data_path, file_path))

# # Combine all data files
# file_list = ['TRD_Week/TRD_Week.csv', 'TRD_Week/TRD_Week1.csv', 'TRD_Week/TRD_Week2.csv']
# # Then delete the .csv file
# TRD = pd.DataFrame()
# for inputfile in file_list:
#     f = open(os.path.join(data_path, inputfile))
#     df_c = pd.read_csv(f)
#     df_m = pd.DataFrame(df_c)
#     TRD = pd.concat([TRD, df_m])

# TRD.reset_index(inplace=True)
# TRD.to_feather(os.path.join(data_path, 'TRD_Week/TRD_Week.ftr'))

file_path = 'TRD_Week/TRD_Week.ftr'
TRD = pd.read_feather(os.path.join(data_path, file_path))

# calculate stock return
TRD['returns'] = TRD['Wclsprc']/TRD['Wopnprc'] - 1

# merge two dataset
TRD_w = TRD_w[TRD_w['Markettype'] == 53]
TRD = pd.merge(TRD, TRD_w[['Trdwnt', 'Cwretmdos']], on='Trdwnt', how='left')

# datetime processing
TRD['Trdwnt'] = TRD['Trdwnt'].str[:4]+TRD['Trdwnt'].str[5:]
TRD['Trdwnt'].astype(int)
TRD['LastDayWeek'] = pd.to_datetime((TRD['Trdwnt']).astype(str)+'6', format="%Y%U%w")
TRD['year'] = TRD['LastDayWeek'].dt.year
TRD['month'] = TRD['LastDayWeek'].dt.month
TRD['YearMonth'] = TRD['year']*100+TRD['month']

# calculate beta
def cal_beta(stock,market):
    beta, alpha, r_value, p_value, std_err = stats.linregress(market, stock)
    return beta

# final factor value
Beta = TRD.groupby(['Stkcd', 'YearMonth']).apply(lambda x: cal_beta(x.returns, x.Cwretmdos))

(6) $bm$: Book-to-market ratio. Data are directly aquired from CSMAR.

(7) $bm\_ia$: The industry-adjusted book-to-market ratio.

In [ ]:
# read file
file_path = 'FI_T10/FI_T10.csv'
FI_t10 = pd.read_csv(os.path.join(data_path, file_path))

# get industry average book-to-market ratio
FI_t10['Ind'] = FI_t10['Indcd'].str[0]
FI_t10.rename(columns={'F101001A': 'bm'}, inplace=True)
df_t10 = FI_t10[['Ind', 'Accper', 'bm']].groupby(['Ind', 'Accper']).mean()

df_t10.reset_index(inplace=True)
df_t10.rename(columns={'bm': 'bm_ind'}, inplace=True)
FI_t10 = pd.merge(FI_t10, df_t10, on=['Ind', 'Accper'], how='left')

# factor values
FI_t10['bm_ia'] = FI_t10['bm'] - FI_t10['bm_ind']

(8) $cash$: Cash and cash equivalents divided by average total assets

**Problem**: CASH contains unexpected number of nulls.

In [ ]:
# Continued with Dataframe: data (see factor 1-3)
data['cash'] = data['CASH']/data['TA']

(9) $cashdebt$: Earnings divided by total liabilities

In [8]:
# Read data
file_path = 'FS_Comins/FS_Comins.csv'
FS_comins = pd.read_csv(os.path.join(data_path, file_path))

FS_comins.rename(columns = {'B001100000': 'Earnings'}, inplace=True)
FS_comins = pd.merge(FS_comins, FS_combas[['Stkcd', 'Accper' ,'CL']], on=['Stkcd', 'Accper'], how='left')

# Calculate factor value
FS_comins = FS_comins['Earnings']/FS_comins['CL']

(60) $pctacc$: Change the denominator with the value of net income. If $NetInc = 0$, then, take $NetInc=0.01$

In [4]:
# Process data
FS_comins = FS_comins[FS_comins['Typrep'] == 'A']
FS_comins.rename(columns = {'B002000000': 'NetInc'}, inplace=True)

# Calculate the characteristic
data = pd.merge(data, FS_comins[['Stkcd', 'Accper', 'NetInc']],
                on = ['Stkcd', 'Accper'],
                how = 'left')
    
data['pctacc'] = ((data['CA'] - data['CA'].shift(1))-
                (data['CASH'] - data['CASH'].shift(1))-
                ((data['CL'] - data['CL'].shift(1))-
                (data['STD'] - data['STD'].shift(1))-
                (data['TP'] - data['TP'].shift(1)))-
                data['Dep'])/\
                data['NetInc']      # Using the same formula as above (except changing the denominator)

(10) $cashspr$: Cash productivity, which is defined as quarter-end       
market capitalization $+$ long-term debt $-$ total assets / cash and equivalents.

In [31]:
# read file
file_path = 'TRD_Mnth/TRD_Mnth.csv'
TRD_m = pd.read_csv(os.path.join(data_path, file_path))

# data process
TRD_m.rename(columns = {'Msmvttl': 'MCap'}, inplace=True) # Market capitalization
## compute quarter-end market capitalization using average
TRD_m['month'] = TRD_m['Trdmnt'].str[5:]
TRD_m['month'] = TRD_m['month'].apply(lambda x: int(x))
TRD_m['quarter'] = TRD_m['month'].apply(lambda x: str(math.ceil(x/3)))
TRD_m['Time'] = TRD_m['Trdmnt'].str[:4] + '-' + TRD_m['quarter']
### compute the average
TRD_m = TRD_m[['Stkcd', 'Time', 'MCap']].groupby(['Stkcd', 'Time']).mean()
## reset index
TRD_m.reset_index(inplace = True)

In [37]:
FS_combas.rename(columns = {'A002206000': 'LD'} # long-term debt
                inplace = True)

#! TO BE CONT'D
#? How to convert the quarter type time to date type time

,Stkcd,Accper,Typrep,CASH,AR,IV,STD,CA,FA,TA,TP,CL,A002206000,ShortName,tang
0,1,2000-01-01,A,8.393075e+09,-393380593.0,NaN,NaN,NaN,1.388777e+09,4.391239e+10,55561410.0,NaN,NaN,深发展A,NaN
1,1,2000-06-30,A,6.852420e+09,429777096.0,520448249.0,NaN,NaN,1.434032e+09,4.973234e+10,55263112.0,NaN,NaN,深发展A,7.444396e+09
2,1,2000-12-31,A,1.044581e+10,-396003224.0,NaN,NaN,NaN,1.587091e+09,6.722750e+10,58298123.0,NaN,NaN,深发展A,NaN
3,1,2001-01-01,A,1.044581e+10,23246566.0,NaN,NaN,NaN,1.587091e+09,6.600617e+10,58298123.0,NaN,NaN,深发展A,NaN
4,1,2001-06-30,A,1.169998e+10,-400131386.0,NaN,NaN,NaN,1.725761e+09,8.518143e+10,49625255.0,NaN,NaN,深发展A,NaN


(83) $tang$: Below is the calculating formula

Cash holdings + 0.75 $\times$ receivables + 0.547 $\times$ inventory + 0.53 $\times$ fixed assets/total assets

In [36]:
FS_combas.rename(columns = {'A001111000': 'AR',
                         'A001123000': 'IV', 
                         'A001212000': 'FA'}, 
                         inplace=True)
FS_combas['tang'] = FS_combas['CASH']+ 0.715*FS_combas['AR'] + 0.547*FS_combas['IV'] + 0.535*FS_combas['FA'] / FS_combas['TA']